In [ ]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

from google.colab import drive
drive.mount("/content/drive")
#Change current working directory to gdrive




Mounted at /content/drive


EDA & Pre-progressing

In [ ]:
#loading data
trainfile = r'/content/drive/MyDrive/CIS508/Assignment_2/bank/Portugese Bank Data _TRAIN.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/content/drive/MyDrive/CIS508/Assignment_2/bank/Portugese Bank Data _TEST.csv'
testData = pd.read_csv(testfile)  #creates a dataframe

print(trainData.shape)
print(testData.shape)


(4521, 17)
(45211, 17)


In [ ]:
#Check trainData set information basic statistics of a data set, column wise
trainData.head()
trainData.info()
trainData.isnull().sum().sum()
trainData.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [ ]:
testData.head()
testData.info()
trainData.isnull().sum().sum()
trainData.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [ ]:
#Check the Column name
trainCols = list(trainData.columns.values)
testCols = list(testData.columns.values)
print(trainCols)
print(testCols)

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']


In [ ]:
# Seperate Target column from Dataset
X_train = trainData.iloc[:, :-1]
X_test = testData.iloc[:, :-1]

Y_train = trainData.iloc[:, -1] 
Y_test = testData.iloc[:, -1] 


In [ ]:
#check the Categorical Features
x = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']
y = ['age',	'balance', 'day	duration',	'campaign',	'pdays',	'previous']

categoricalFeatures = [item for item in x if item not in y]

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([X_train,X_test], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)


In [ ]:
print(Y_train.value_counts('Yes'))
print(Y_test.value_counts('Yes'))

no     0.88476
yes    0.11524
Name: y, dtype: float64
no     0.883015
yes    0.116985
Name: y, dtype: float64


Decision Tree Classifier

In [ ]:
#Decision Tree Classifier ========================================================================
clf = DecisionTreeClassifier(random_state=1)
clf.fit(X_train, Y_train)
clf_pred=clf.predict(X_train)
clfr_pred=clf.predict(X_test)

In [ ]:
#TrainData Score check
print("accuracy Score (trainset) :{0:6f}".format(clf.score(X_train,Y_train))) 
print("Confusion Matrix")
print(confusion_matrix(Y_train,clf_pred))
print("=== Classification Report ===")
print(classification_report(Y_train, clf_pred))
clf_cv_score = cross_val_score(clf, X_train, Y_train)
print(clf_cv_score)
print('\n')

#test Score check
clf = DecisionTreeClassifier(random_state=1)
clf.fit(X_train,Y_train)
clf_pred = clf.predict(X_test)
print("accuracy Score (testset):{0:6f}".format(clf.score(X_test,Y_test)))
print("Confusion Matrix")
print(confusion_matrix(Y_test,clfr_pred))

print("=== Classification Report ===")
print(classification_report(Y_test, clf_pred))
clf_cv_score = cross_val_score(clf, X_test, Y_test)
print(clf_cv_score)
print('\n')


accuracy Score (trainset) :1.000000
Confusion Matrix
[[4000    0]
 [   0  521]]
=== Classification Report ===
              precision    recall  f1-score   support

          no       1.00      1.00      1.00      4000
         yes       1.00      1.00      1.00       521

    accuracy                           1.00      4521
   macro avg       1.00      1.00      1.00      4521
weighted avg       1.00      1.00      1.00      4521

[0.86629834 0.87168142 0.87057522 0.87278761 0.86615044]


accuracy Score (testset):0.882816
Confusion Matrix
[[38174  1748]
 [ 3550  1739]]
=== Classification Report ===
              precision    recall  f1-score   support

          no       0.91      0.96      0.94     39922
         yes       0.50      0.33      0.40      5289

    accuracy                           0.88     45211
   macro avg       0.71      0.64      0.67     45211
weighted avg       0.87      0.88      0.87     45211

[0.82948137 0.09876134 0.25204601 0.2366733  0.20415837]




Hyperparameter tuning for dicision Tree and each model evaluation

In [ ]:
#RandomizedSearchCV(1)
clf1 = DecisionTreeClassifier(random_state=1)
parameters1={"max_depth": range(1,50,2),"min_samples_leaf" : range(1,100,10),'max_leaf_nodes': range(1,100,1),'class_weight':['balanced','none']}
clf_random1 = RandomizedSearchCV(clf1,parameters1,scoring= 'balanced_accuracy',n_iter=10,cv=5)  
clf_random1.fit(X_train, Y_train)
grid_parm1=clf_random1.best_params_
print(grid_parm1)

{'min_samples_leaf': 10, 'max_leaf_nodes': 19, 'max_depth': 26, 'class_weight': 'balanced'}


In [ ]:
#Construct Decision Trees using the RandomizedSearchCV result 
clf1 = DecisionTreeClassifier(random_state=1, min_samples_leaf=10, max_leaf_nodes=19, max_depth=26, class_weight='balanced') 
clf1.fit(X_train,Y_train)
clf_pred1 = clf1.predict(X_test)

print("accuracy Score (testset):{0:6f}".format(clf1.score(X_test,Y_test)))
print("Confusion Matrix")
print(confusion_matrix(Y_test,clf_pred1))


print("=== Classification Report ===")
print(classification_report(Y_test,clf_pred1))
clf_cv_score = cross_val_score(clf1, X_train, Y_train, cv=10, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')

accuracy Score (testset):0.611886
Confusion Matrix
[[23844 16078]
 [ 1469  3820]]
=== Classification Report ===
              precision    recall  f1-score   support

          no       0.94      0.60      0.73     39922
         yes       0.19      0.72      0.30      5289

    accuracy                           0.61     45211
   macro avg       0.57      0.66      0.52     45211
weighted avg       0.85      0.61      0.68     45211

[0.6853066  0.62471154 0.58423077 0.62596154 0.64875    0.62740385
 0.60932692 0.63567308 0.64365385 0.60548077]




In [ ]:
#RandomizedSearchCV(2)
clf2 = DecisionTreeClassifier(random_state=1)
parameters2={'min_samples_leaf' : range(10,20,1),'max_depth':range(15,25,1),'max_leaf_nodes': range(15,23,1),'class_weight':['balanced','none']}

clf_random2 = RandomizedSearchCV(clf2,parameters2,n_iter=10,cv=5,scoring= 'balanced_accuracy')  
clf_random2.fit(X_train, Y_train)
grid_parm2=clf_random2.best_params_
print(grid_parm2)

{'min_samples_leaf': 16, 'max_leaf_nodes': 17, 'max_depth': 19, 'class_weight': 'balanced'}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier
#Construct Decision Trees using the best parameters
clf2 = DecisionTreeClassifier(random_state=1, min_samples_leaf=16, max_leaf_nodes=17, max_depth=19, class_weight='balanced') 
clf2.fit(X_train,Y_train)
clf_pred2 = clf2.predict(X_test)

print("accuracy Score (testset):{0:6f}".format(clf2.score(X_test,Y_test)))
print("Confusion Matrix")
print(confusion_matrix(Y_test,clf_pred2))

print("=== Classification Report ===")
print(classification_report(Y_test,clf_pred2))
clf_cv_score = cross_val_score(clf2, X_train, Y_train, cv=10, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')

accuracy Score (testset):0.591936
Confusion Matrix
[[22762 17160]
 [ 1289  4000]]
=== Classification Report ===
              precision    recall  f1-score   support

          no       0.95      0.57      0.71     39922
         yes       0.19      0.76      0.30      5289

    accuracy                           0.59     45211
   macro avg       0.57      0.66      0.51     45211
weighted avg       0.86      0.59      0.66     45211

[0.66292453 0.62846154 0.66932692 0.62846154 0.62230769 0.63663462
 0.66269231 0.64028846 0.66163462 0.61115385]




In [ ]:
#Grid search
clf = DecisionTreeClassifier(random_state=1)
clf_grid = GridSearchCV(clf,parameters)
clf_grid.fit(X_train, Y_train)
grid_parm_g1=clf_grid.best_params_ 
print(grid_parm_g1)

{'class_weight': 'balanced', 'max_depth': 1, 'max_leaf_nodes': 15, 'min_samples_leaf': 2}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier
#Construct Decision Trees using the best parameters
clfr = DecisionTreeClassifier(random_state=1, min_samples_leaf=2, max_leaf_nodes=15, max_depth=1, class_weight='balanced') 
clfr.fit(X_train,Y_train)
clfr_pred = clfr.predict(X_test)

print("accuracy Score (testset):{0:6f}".format(clfr.score(X_test,Y_test)))
print("Confusion Matrix")
print(confusion_matrix(Y_test,clf_pred2))

print("=== Classification Report ===")
print(classification_report(Y_test,clfr_pred))
clf_cv_score = cross_val_score(clfr, X_train, Y_train, cv=10, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')

accuracy Score (testset):0.892858
Confusion Matrix
[[22762 17160]
 [ 1289  4000]]
=== Classification Report ===
              precision    recall  f1-score   support

          no       0.90      0.99      0.94     39922
         yes       0.65      0.18      0.29      5289

    accuracy                           0.89     45211
   macro avg       0.77      0.59      0.61     45211
weighted avg       0.87      0.89      0.87     45211

[0.57740566 0.57192308 0.58028846 0.58865385 0.56105769 0.56355769
 0.54557692 0.57903846 0.60076923 0.57067308]




Random forest

In [ ]:
#randomforest==============================================================================

rfc = RandomForestClassifier()  
rfc.fit(X_train, Y_train)
rfc_pred=rfc.predict(X_test)

print("accuracy Score (testset) for RandomForest:{0:6f}".format(rfc.score(X_test,Y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(Y_test,rfc_pred))
print("=== Classification Report ===")
print(classification_report(Y_test,rfc_pred))
rfc_cv_score = cross_val_score(rfc, X_train, Y_train, cv=10)
print(rfc_cv_score)
print('\n')


accuracy Score (testset) for RandomForest:0.901794
Confusion Matrix for Random Forest:
[[39663   259]
 [ 4181  1108]]
=== Classification Report ===
              precision    recall  f1-score   support

          no       0.90      0.99      0.95     39922
         yes       0.81      0.21      0.33      5289

    accuracy                           0.90     45211
   macro avg       0.86      0.60      0.64     45211
weighted avg       0.89      0.90      0.88     45211

[0.89183223 0.89823009 0.89159292 0.88938053 0.88495575 0.88938053
 0.89159292 0.88716814 0.89380531 0.88938053]




Hyperparameter tuning for Random forest

In [ ]:
#Random_search
rfc = RandomForestClassifier(random_state= 1)
rand_parameters={"n_estimators" : [20,40,60,80,100],"max_depth": range(1, 10, 2),"min_samples_leaf" : range(1,20,5),'max_leaf_nodes': [15,20,25,30,35], 'class_weight':['balanced','none']}
rfc_random = RandomizedSearchCV(rfc,rand_parameters,n_iter=15,cv=5)
rfc_random.fit(X_train, Y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

{'n_estimators': 100, 'min_samples_leaf': 1, 'max_leaf_nodes': 35, 'max_depth': 5, 'class_weight': 'balanced'}


In [ ]:
#Grid search
rfcr = RandomForestClassifier(random_state= 1)
rfcr_grid = GridSearchCV(rfcr,rand_parameters)
rfcr_grid.fit(X_train, Y_train)
grid_parm_rfcr =rfcr_grid.best_params_
print(grid_parm_rfcr)

{'class_weight': 'balanced', 'max_depth': 9, 'max_leaf_nodes': 35, 'min_samples_leaf': 1, 'n_estimators': 100}


In [ ]:
#create new classifier using the best parameters
rfc= RandomForestClassifier(n_estimators=100, min_samples_leaf=1, max_leaf_nodes=35, max_depth=5, class_weight= 'balanced')
rfcr= RandomForestClassifier(n_estimators=100, min_samples_leaf=1, max_leaf_nodes=35, max_depth=9, class_weight= 'balanced')

rfc.fit(X_train,Y_train)
rfc_pred = rfc.predict(X_test)
rfcr.fit(X_train,Y_train)
rfcr_pred = rfcr.predict(X_test)

Model Evaluation

In [ ]:
#Obtain accuracy ,confusion matrix,classification report 
print("accuracy Score (testset) after hypertuning randomized search for Random Forest:{0:6f}".format(rfc.score(X_test,Y_test)))
print("accuracy Score (testset) after hypertuning grid search for Random Forest:{0:6f}".format(rfcr.score(X_test,Y_test)))
print("=== Confusion Matrix ===")
print("randomsearch")
print(confusion_matrix(Y_test,rfc_pred))
print('\n')
print("Gridsearch")
print(confusion_matrix(Y_test,rfcr_pred))
print('\n')
print("=== Classification Report ===")
print("randomsearch")
print(classification_report(Y_test,rfc_pred))
print('\n')
print("Gridsearch")
print(classification_report(Y_test,rfcr_pred))
print('\n')
rfc_cv_score = cross_val_score(rfc, X_train, Y_train)
print(rfc_cv_score)
print('\n')

accuracy Score (testset) after hypertuning randomized search for Random Forest:0.742585
accuracy Score (testset) after hypertuning grid search for Random Forest:0.793988
=== Confusion Matrix ===
randomsearch
[[30254  9668]
 [ 1970  3319]]


Gridsearch
[[32795  7127]
 [ 2187  3102]]


=== Classification Report ===
randomsearch
              precision    recall  f1-score   support

          no       0.94      0.76      0.84     39922
         yes       0.26      0.63      0.36      5289

    accuracy                           0.74     45211
   macro avg       0.60      0.69      0.60     45211
weighted avg       0.86      0.74      0.78     45211



Gridsearch
              precision    recall  f1-score   support

          no       0.94      0.82      0.88     39922
         yes       0.30      0.59      0.40      5289

    accuracy                           0.79     45211
   macro avg       0.62      0.70      0.64     45211
weighted avg       0.86      0.79      0.82     45211



[0.